In [10]:
%load_ext autoreload
%autoreload 2
%cd ../..

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/chenyenpin/Documents/ADAPR5


# Importando bibliotecas

In [33]:
import pandas as pd
import numpy as np
import requests
import os

from dotenv import (find_dotenv,
                    load_dotenv)
from sklearn.model_selection import train_test_split as tests
from sklearn.linear_model import (LinearRegression)

from sklearn.ensemble import (RandomForestRegressor,
                              GradientBoostingRegressor,
                              AdaBoostRegressor,
                              BaggingRegressor,
                              VotingRegressor)
from sklearn import metrics

from sklearn.preprocessing import (MinMaxScaler, 
                                   StandardScaler,
                                   RobustScaler,
                                   PowerTransformer)
import warnings

pd.set_option("display.max_rows",1000)

warnings.filterwarnings("ignore")

In [34]:
def eda(df:pd.DataFrame)->None:
  print("-"*15,"DataFrame shape", "-"*15 )
  print(f"Linhas: {df.shape[0]}, Colunas: {df.shape[1]}")
  print("-"*15,"DataFrame info", "-"*15 )
  print(df.info())
  print("-"*15,"DataFrame isna", "-"*15 )
  print(df.isna().sum())
  print("-"*15,"DataFrame is duplicated", "-"*15 )
  print(df.duplicated().sum())
  print("-"*15,"DataFrame describe numeric", "-"*15 )
  print(df.describe(percentiles=[.01,0.1,.25,.5,.75,.90,.99]))
  print("-"*15,"DataFrame desribe categorical", "-"*15 )
  print(df.describe(exclude=[np.number]))




def date_engineering(df: pd.DataFrame)->pd.DataFrame:
    df["data_inversa"]=pd.to_datetime(df["data_inversa"],format='mixed')
    df["year"]=pd.to_datetime(df["data_inversa"]).dt.year
    df["month"]=pd.to_datetime(df["data_inversa"]).dt.month
    df["quarter"]=pd.to_datetime(df["data_inversa"]).dt.quarter
    df["quarter_end"]=pd.to_datetime(df["data_inversa"]).dt.is_quarter_end
    return df


def date_col(col:pd.Series):
  df["dia_semana"]=df['dia_semana'].str.split("-").str[0]
  return df["dia_semana"]

def time_col(col:pd.Series):
  df["hour"] = df["horario"].str.split(":").str[0]

  return df["hour"]

def convert_lower_case(column):
  '''
  Funcao que retorna a coluna selecionada para letra minuscula
  -param column: coluna da pd.Dataframe
  '''
  return column.lower()



In [13]:
load_dotenv(find_dotenv())
path=os.getenv("path")


In [14]:
df=pd.read_csv(path)
df.head()

,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,...,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
0,10/11/2009,Terça,14:10:00,MG,381.0,495.0,BETIM,Velocidade incompatível,Colisão traseira,Com Vítimas Feridas,...,0,1,0,4,2,NaN,NaN,NaN,NaN,NaN
1,16/10/2009,Sexta,04:35:00,MG,251.0,419.6,GRAO MOGOL,Outras,Colisão frontal,Com Vítimas Fatais,...,5,0,0,5,2,NaN,NaN,NaN,NaN,NaN
2,20/09/2009,Domingo,20:00:00,PI,316.0,24.0,TERESINA,Dormindo,Saída de Pista,Com Vítimas Feridas,...,4,0,0,6,1,NaN,NaN,NaN,NaN,NaN
3,17/05/2009,Domingo,15:45:00,RJ,101.0,429.0,MANGARATIBA,Falta de atenção,Colisão Transversal,Sem Vítimas,...,0,2,0,0,2,NaN,NaN,NaN,NaN,NaN
4,05/07/2009,Domingo,17:30:00,PE,101.0,59.2,RECIFE,Outras,Queda de motocicleta / bicicleta / veículo,Com Vítimas Feridas,...,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN


In [ ]:
# transformando e criando colunas adicionais para data
df=date_engineering(df)
df["hour"]=time_col(df["horario"])

In [ ]:
df=df.drop(columns=["latitude","longitude",
                    "regional","delegacia","uop","ano","municipio"])

In [ ]:
df["date"]=pd.to_datetime(df["data_inversa"])
df["date"].info()

In [ ]:
cat_col=df.select_dtypes(exclude="number").columns.to_list()
num_col=[col for col in df if col not in cat_col]

In [ ]:
num_col=[
    #'br',
 'pessoas',
 #'mortos',
 'feridos_leves',
 'feridos_graves',
 'ilesos',
 'ignorados',
 'feridos',
 'veiculos',
 'year',
 'month',
 'quarter']

In [ ]:
cat_col=[
  #'data_inversa',
 #'dia_semana',
 #'horario',
 'uf',
 #'km',
 #'municipio',
 #'causa_acidente',
 #'tipo_acidente',
 'classificacao_acidente',
 'fase_dia',
 'sentido_via',
 'condicao_metereologica',
 'tipo_pista',
 'tracado_via',
 'uso_solo',
 'hour',
  ]